In [1]:
%pip install undetected-chromedriver
%pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## TRYING ARXIV (FAILED)
## PLEASE SCROLL DOWN TO WEB SCRAPER

In [ ]:
import requests
import os
os.chdir("D://ugpold")
url = 'https://export.arxiv.org/api/query?id_list=2201.00583'


response = requests.get(url)

if response.status_code == 200:
    with open('arxiv_metadata.xml', 'wb') as file:
        file.write(response.content)
    print("XML saved as arxiv_metadata.xml")
else:
    print(f"Failed to fetch data, status code: {response.status_code}")


XML saved as arxiv_metadata.xml


In [2]:
from bs4 import BeautifulSoup
import json
import os
import time
import undetected_chromedriver as uc
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [3]:
%pip install requests PyPDF2 pdfplumber

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import requests
import PyPDF2
from io import BytesIO
import pandas as pd

In [ ]:
def save_full_journal_to_text(pdf_address, pdf_reader):
    safe_pdf_address = pdf_address.replace("http://", "").replace("https://", "").replace("/", "_")
    
    full_text_file_name = f"{safe_pdf_address}_full.txt"
    

    full_journal_text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        if text:
            full_journal_text += text 

    with open(full_text_file_name, "w", encoding="utf-8") as full_text_file:
        full_text_file.write(full_journal_text)

    print(f"Full journal saved to {full_text_file_name}")



In [ ]:
def clean_text(text):
    """Remove illegal characters that cannot be used in Excel."""
    return ''.join(c for c in text if c.isprintable())

def save_to_text_file(pdf_address, data):
    safe_pdf_address = pdf_address.replace("http://", "").replace("https://", "").replace("/", "_")

    text_file_name = f"{safe_pdf_address}.txt"

    with open(text_file_name, "w", encoding="utf-8") as text_file:
        for journal_number, paragraph_number, paragraph_text in data:
            text_file.write(f"Journal Number: {journal_number}, Paragraph Number: {paragraph_number}\n")
            text_file.write(f"{paragraph_text}\n\n")

    print(f"Data saved to {text_file_name}")

def findJournalContent(pdf_url, pdf_address, file="pld_journals.xlsx"):
    driver = webdriver.Chrome()
    driver.get(pdf_url)

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "embed"))
        )
        print(f"PDF found: {pdf_url}")

        response = requests.get(pdf_url, timeout=10)
        if response.status_code == 200:
            pdf_file = BytesIO(response.content)
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            
            data = []
            paragraph_number = 1

            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                if text:
                    paragraphs = text.replace(".\n", ".\n-\n").split("\n-\n")
                    for para in paragraphs:
                        cleaned_para = clean_text(para.strip())
                        if cleaned_para:
                            data.append([pdf_address, paragraph_number, cleaned_para])
                            paragraph_number += 1
            
            df = pd.DataFrame(data, columns=["Journal Number", "Paragraph Number", "Paragraph Text"])

            try:
                existing_df = pd.read_excel(file)
                df = pd.concat([existing_df, df], ignore_index=True)
            except FileNotFoundError:
                pass  

            df.to_excel(file, index=False)
            print(f"Data saved to {file}")


            save_to_text_file(pdf_address, data)
    
            save_full_journal_to_text(pdf_address, pdf_reader)


        else:
            print("Failed to download the PDF. Status code:", response.status_code)

    except TimeoutException:
        print("Unable to find the PDF within 10 seconds.")
    except NoSuchElementException:
        print("PDF element not found on the page.")
    finally:
        driver.quit()



In [8]:
def find_arxiv_pdf_links_dict(query, pages=1):
    base_url = "https://arxiv.org/search/"
    pdf_links_dict = {}

    for start in range(0, pages * 50, 50):
        url = f"{base_url}?query={query}&searchtype=all&source=header&start={start}"
        print(f"Scraping page: {url}")
        
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch page {start // 50 + 1}. Status Code: {response.status_code}")
            continue
    
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a', href=lambda href: href and href.startswith("https://arxiv.org/pdf/"))
        
        for link in links:
            href = link['href']
            pdf_address = href.split('/')[-1].replace('.', '_')
            pdf_links_dict[pdf_address] = href

    return pdf_links_dict


In [11]:
query = "pulsed laser deposition"
query=query.replace(" ","+")
pdf_links_dict = find_arxiv_pdf_links_dict(query, pages=1)

print(f"{len(pdf_links_dict)} PDF links scraped:")

cnt = 0
for key, link in pdf_links_dict.items():
    print(f"{key}: {link}")
    cnt += 1
    if cnt == 5:
        break

Scraping page: https://arxiv.org/search/?query=pulsed+laser+deposition&searchtype=all&source=header&start=0
50 PDF links scraped:
2502_18938: https://arxiv.org/pdf/2502.18938
2502_16168: https://arxiv.org/pdf/2502.16168
2502_12612: https://arxiv.org/pdf/2502.12612
2502_09382: https://arxiv.org/pdf/2502.09382
2502_00809: https://arxiv.org/pdf/2502.00809


In [13]:
for key, link in pdf_links_dict.items():
    findJournalContent(link, key)

PDF found: https://arxiv.org/pdf/2502.18938
Data saved to pld_journals.xlsx
Data saved to 2502_18938.txt
Full journal saved to 2502_18938_full.txt
PDF found: https://arxiv.org/pdf/2502.16168
Data saved to pld_journals.xlsx
Data saved to 2502_16168.txt
Full journal saved to 2502_16168_full.txt
PDF found: https://arxiv.org/pdf/2502.12612
Data saved to pld_journals.xlsx
Data saved to 2502_12612.txt
Full journal saved to 2502_12612_full.txt
PDF found: https://arxiv.org/pdf/2502.09382
Data saved to pld_journals.xlsx
Data saved to 2502_09382.txt
Full journal saved to 2502_09382_full.txt
PDF found: https://arxiv.org/pdf/2502.00809
Data saved to pld_journals.xlsx
Data saved to 2502_00809.txt
Full journal saved to 2502_00809_full.txt
PDF found: https://arxiv.org/pdf/2501.18523
Data saved to pld_journals.xlsx
Data saved to 2501_18523.txt
Full journal saved to 2501_18523_full.txt
PDF found: https://arxiv.org/pdf/2501.16189
Data saved to pld_journals.xlsx
Data saved to 2501_16189.txt
Full journal 

incorrect startxref pointer(3)


Data saved to pld_journals.xlsx
Data saved to 2411_16059.txt
Full journal saved to 2411_16059_full.txt
PDF found: https://arxiv.org/pdf/2411.16047
Data saved to pld_journals.xlsx
Data saved to 2411_16047.txt
Full journal saved to 2411_16047_full.txt
PDF found: https://arxiv.org/pdf/2411.15023
Data saved to pld_journals.xlsx
Data saved to 2411_15023.txt
Full journal saved to 2411_15023_full.txt
PDF found: https://arxiv.org/pdf/2411.10073
Data saved to pld_journals.xlsx
Data saved to 2411_10073.txt
Full journal saved to 2411_10073_full.txt
PDF found: https://arxiv.org/pdf/2411.02346
Data saved to pld_journals.xlsx
Data saved to 2411_02346.txt
Full journal saved to 2411_02346_full.txt
PDF found: https://arxiv.org/pdf/2410.23895
Data saved to pld_journals.xlsx
Data saved to 2410_23895.txt
Full journal saved to 2410_23895_full.txt
PDF found: https://arxiv.org/pdf/2410.23185
Data saved to pld_journals.xlsx
Data saved to 2410_23185.txt
Full journal saved to 2410_23185_full.txt
PDF found: http

In [52]:
os.getcwd()

'd:\\UGP'

# WEB SCRAPER

### Note:
This project does not endorse or promote unauthorized web scraping. The data collection was limited to a small number of publicly accessible pages (maximum 5 pages) solely for experimental and educational purposes. 
All content retrieved was accessed using official APIs or publicly available endpoints, where permitted. 
Please ensure compliance with the terms of service of any website or API used.

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import undetected_chromedriver as uc

driver=uc.Chrome()

all_links = []
for i in range(5):

    url = f"https://link.springer.com/search?new-search=true&query=pulsed+laser+deposition&sortBy=relevance&page={i+1}"
    driver.get(url)
    time.sleep(3)
    page_source = driver.page_source

    soup = BeautifulSoup(page_source, "html.parser")

    links = soup.find_all("a", href=True)
    article_links = [link['href'] for link in links if link['href'].startswith('/article/')]
    
    all_links.append(article_links)
    for article_link in article_links:
        print(article_link)

driver.quit()


/article/10.1007/s11468-024-02706-6
/article/10.1007/s40964-024-00855-y
/article/10.1007/s40516-024-00255-8
/article/10.1007/s00170-024-14899-5
/article/10.1007/s12596-024-01870-6
/article/10.1134/S1063782625700010
/article/10.1007/s12596-024-02150-z
/article/10.1007/s10854-024-13485-5
/article/10.1007/s12596-024-01951-6
/article/10.1007/s12596-025-02483-3
/article/10.1007/s10450-024-00500-8
/article/10.1007/s00339-024-07799-8
/article/10.1007/s10854-025-14462-2
/article/10.1007/s10854-023-11601-5
/article/10.1007/s11664-024-11322-3
/article/10.1038/s41598-023-47955-3
/article/10.1038/s41524-024-01275-w
/article/10.1007/s10854-023-11601-5
/article/10.1038/s41598-023-47955-3
/article/10.1007/s11082-023-05912-4
/article/10.1007/s12596-025-02478-0
/article/10.1007/s12596-023-01587-y
/article/10.1007/s40964-024-00629-6
/article/10.1007/s00339-023-06549-6
/article/10.1007/s42864-023-00235-z
/article/10.1007/s11468-024-02234-3
/article/10.1007/s11468-023-02016-3
/article/10.1007/s11082-023-0

In [ ]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import time
import os

os.makedirs("D://testing", exist_ok=True)
os.chdir("D://testing")
driver = uc.Chrome()

os.makedirs("downloaded_articles", exist_ok=True)

for page in all_links:
    for article_url in page:
        try:
            name = article_url.replace(".", "_").replace("/", "_")
            print(name)
            article_url="https://link.springer.com"+article_url
            driver.get(article_url)
            time.sleep(3)
            page_source = driver.page_source
            
            soup = BeautifulSoup(page_source, "html.parser")
       
            filename = f"downloaded_articles/{name}.html"
            
            with open(filename, "w", encoding="utf-8") as file:
                file.write(str(soup))
            
            print(f"Downloaded: {article_url}")
            print(f"Path: D://testing/{filename}")
        
        except Exception as e:
            print(f"Failed to download {article_url}: {e}")

driver.quit()


_article_10_1007_s11468-024-02706-6
Downloaded: https://link.springer.com/article/10.1007/s11468-024-02706-6
Path: D://testing/downloaded_articles/_article_10_1007_s11468-024-02706-6.html
_article_10_1007_s40964-024-00855-y
Downloaded: https://link.springer.com/article/10.1007/s40964-024-00855-y
Path: D://testing/downloaded_articles/_article_10_1007_s40964-024-00855-y.html
_article_10_1007_s40516-024-00255-8
Downloaded: https://link.springer.com/article/10.1007/s40516-024-00255-8
Path: D://testing/downloaded_articles/_article_10_1007_s40516-024-00255-8.html
_article_10_1007_s00170-024-14899-5
Downloaded: https://link.springer.com/article/10.1007/s00170-024-14899-5
Path: D://testing/downloaded_articles/_article_10_1007_s00170-024-14899-5.html
_article_10_1007_s12596-024-01870-6
Downloaded: https://link.springer.com/article/10.1007/s12596-024-01870-6
Path: D://testing/downloaded_articles/_article_10_1007_s12596-024-01870-6.html
_article_10_1134_S1063782625700010
Downloaded: https://link.

In [ ]:
from bs4 import BeautifulSoup
import os
import json

folder_path = 'D:/testing/downloaded_articles'


data = {}

for i in range(50):
    filename = f'paper_{i}.html'
    file_path = os.path.join(folder_path, filename)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        

        main_content = soup.find('div', class_='main-content')
        
        paragraphs = []
        if main_content:
            for p in main_content.find_all('p'):
                text = p.get_text(strip=True)
                if text:
                    paragraphs.append(text)

        data[f'paper_{i}'] = paragraphs


output_path = os.path.join(folder_path, 'paragraphs_data.json')
with open(output_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, indent=2, ensure_ascii=False)


In [46]:
from bs4 import BeautifulSoup
import os
import json

folder_path = 'D:/testing/downloaded_articles'

data = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.html'):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'r', encoding='utf-8') as file:
            soup = BeautifulSoup(file, 'html.parser')
            
            main_content = soup.find('div', class_='main-content')
            
            paragraphs = []
            if main_content:
                for p in main_content.find_all('p'):
                    text = p.get_text(strip=True)
                    if text:
                        paragraphs.append(text)

            file_key = filename.replace('_article_', '/article/').replace('_s','/s').replace('_S','/S')

            file_key = file_key.replace('_','.')
            file_key = "https://link.springer.com" + file_key
            print(file_key[:-5]) #removing html
            data[file_key[:-5]] = paragraphs

output_path = os.path.join(folder_path, 'paragraphs_data.json')
with open(output_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, indent=2, ensure_ascii=False)


https://link.springer.com/article/10.1007/s00170-023-11238-y
https://link.springer.com/article/10.1007/s00170-024-13455-5
https://link.springer.com/article/10.1007/s00170-024-14514-7
https://link.springer.com/article/10.1007/s00170-024-14899-5
https://link.springer.com/article/10.1007/s00170-024-14991-w
https://link.springer.com/article/10.1007/s00339-022-05698-4
https://link.springer.com/article/10.1007/s00339-022-06166-9
https://link.springer.com/article/10.1007/s00339-022-06319-w
https://link.springer.com/article/10.1007/s00339-023-06508-1
https://link.springer.com/article/10.1007/s00339-023-06549-6
https://link.springer.com/article/10.1007/s00339-023-07199-4
https://link.springer.com/article/10.1007/s00339-024-07571-y
https://link.springer.com/article/10.1007/s00339-024-07674-6
https://link.springer.com/article/10.1007/s00339-024-07799-8
https://link.springer.com/article/10.1007/s00339-024-08149-4
https://link.springer.com/article/10.1007/s00339-025-08403-3
https://link.springer.co

In [ ]:
import json
import csv
os.chdir("D://testing")
json_path = 'D:/testing/downloaded_articles/paragraphs_data.json'
csv_path = 'D:/testing/para_data.csv'

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)


with open(csv_path, 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    #header row
    writer.writerow(['link', 'paper', 'paragraph_index', 'text'])


    for file_name, paragraphs in data.items():
        link = file_name  #'file_name' == 'link'
        for idx, para in enumerate(paragraphs):
            writer.writerow([link, file_name, idx, para])
